In [2]:
from crewai import Agent, Crew, Process, Task, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool, TXTSearchTool
from crewai_tools import FileReadTool, CSVSearchTool
from crewai.tools import BaseTool
from bs4 import BeautifulSoup
import requests
from crewai.tools import tool
import json
import pandas as pd
import json
import os

In [3]:
from langchain.llms import Ollama
llm=LLM(model="ollama/llama3", base_url="http://localhost:11434")

### Tools

In [4]:
current_directory = os.getcwd()

In [5]:
current_directory

'/Users/derekwu/Desktop/seldon-labs/notebooks'

In [ ]:
# Read the Excel file
cpd_df = pd.read_excel('cpd_documents.xlsx')

# Convert the DataFrame to JSON
json_data = cpd_df.to_json(orient='records',indent=4)

# Write the JSON data to a file
with open('cpd_documents.json', 'w') as f:
    f.write(json_data)

In [ ]:
# Read the Excel file
news_df = pd.read_excel("news_dataset.xlsx")

# Convert the DataFrame to JSON
json_data = news_df.to_json(orient='records',indent=4)

# Write the JSON data to a file
with open('news_dataset.json', 'w') as f:
    f.write(json_data)

In [ ]:
file_tool = FileReadTool()

### Agents

In [6]:
agents = {'game theory expert': {'goal':  """Analyze the all of the compiled research  """ , 
                                 'backstory': """You are a world-renowed game theory expert. Your speciality is studying geopolitical events and looking at economic and 
                                                political factors to determine outcomes and payoffs for potential game theory scenarios"""
                                },
          'news researcher': {'goal': f"""Analyze the news articles in {current_directory}/cpd_documents.json and all the tariff related
                                            government statements in {current_directory}/cpd_documents.json to model 
                                            the outcomes for the United States and Canada if tariffs are enacted""", 
                                 'backstory': """You are a world-renowed game theory expert. Your speciality is studying geopolitical events and looking at economic and 
                                                political factors to determine outcomes and payoffs for potential game theory scenarios"""
                             },
          'political researcher': {'goal': f"""Analyze the news articles in {current_directory}/cpd_documents.json and all the tariff related
                                                government statements in {current_directory}/cpd_documents.json to model 
                                                the outcomes for the United States and Canada if tariffs are enacted""", 
                                   'backstory': """You are a world-renowed game theory expert. Your speciality is studying geopolitical events and 
                                                looking at economic and political factors to determine outcomes and payoffs for potential game theory scenarios"""
                             }
         }

In [ ]:
game_theory_agent = Agent(role="game theory expert",
                          goal=agents['game theory expert']['goal'],
                          backstory=agents['game theory expert']['goal'],
                          tools=[file_tool],
                          llm=llm,
                         )

### Tasks

In [ ]:
task = {'payoff_modeling': {'description': """Analyze all of recent news and official statements by the US administration and 
                                              predict the payoffs for the four different scenarios [(no tariff, no tariff), (no tariff, tariff)
                                              (tariff, no tariff), (tariff, tariff)] of a trade war game between Canada and the US""",
                            'expected output': """For each of the four scenarios generate an economic payoff, international relations payoff, and 
                                                  domestic relations payoff for each player. The payoff should be between -50 and 50. After generating
                                                  the payoffs, write a quicky summary explaining your reasoning.
                                               """ 
                           },
        'news_research': {'description': """Analyze all of recent news and official statements by the US administration and 
                                              predict the payoffs for the four different scenarios [(no tariff, no tariff), (no tariff, tariff)
                                              (tariff, no tariff), (tariff, tariff)] of a trade war game between Canada and the US""",
                            'expected output': """For each of the four scenarios generate an economic payoff, international relations payoff, and 
                                                  domestic relations payoff for each player. The payoff should be between -50 and 50. After generating
                                                  the payoffs, write a quicky summary explaining your reasoning.
                                               """ 
                           },
        'political_research': {'description': """Analyze all of recent news and official statements by the US administration and 
                                              predict the payoffs for the four different scenarios [(no tariff, no tariff), (no tariff, tariff)
                                              (tariff, no tariff), (tariff, tariff)] of a trade war game between Canada and the US""",
                            'expected output': """For each of the four scenarios generate an economic payoff, international relations payoff, and 
                                                  domestic relations payoff for each player. The payoff should be between -50 and 50. After generating
                                                  the payoffs, write a quicky summary explaining your reasoning.
                                               """ 
                           }
       }


In [ ]:
payoff_task = Task(description=task['payoff_modeling']['description'],
                   expected_output=task['payoff_modeling']['expected output'],
                   agent=game_theory_agent
)

In [ ]:
game_theory_crew = Crew(agents=[game_theory_agent],
                        tasks=[payoff_task],
                        process=Process.sequential,
                        verbose=True
)


In [ ]:
result = game_theory_crew.kickoff()
print("Crew Execution Complete. Final report generated.")
print(result)